In [126]:
import requests
import pandas as pd


from app import *


In [127]:
gateio_tokens = get_gateio_pre_market_currencies()
bybit_tokens = get_bybit_pre_market_currencies()

In [145]:
gateio_tokens.columns

Index(['currency', 'full_name', 'icon_link', 'total_num_issued',
       'white_book_link', 'official_website_link', 'desc', 'is_hot',
       'status_code', 'status', 'price_precision', 'volume_precision',
       'amount_precision', 'min_order_amount', 'max_order_amount',
       'min_order_value', 'max_order_value', 'trade_start_time',
       'trade_end_time', 'delivery_start_time', 'delivery_end_time', 'margin',
       'buy_maker_fee', 'buy_taker_fee', 'sell_maker_fee', 'sell_taker_fee',
       'latest_deal_price', 'highest_buy_price', 'lowest_sell_price'],
      dtype='object')

In [128]:
common_tokens = list(set(gateio_tokens['currency'].unique().tolist()) & set(bybit_tokens['tokenName'].unique().tolist()))

In [129]:
common_tokens

['EIGEN', 'IO']

In [130]:
token = common_tokens[0]

In [131]:
bybit_bids = get_bybit_pre_market_orderbook(token, 'buy')
bybit_asks = get_bybit_pre_market_orderbook(token, 'sell')

In [132]:
gateio_bids = get_gateio_pre_market_orderboook(token, 'buy')
gateio_asks = get_gateio_pre_market_orderboook(token, 'sell')

In [133]:
def convert_orderbook_tick(orderbook_df: pd.DataFrame, exchange: str, spread: float = 0.0):
    quotes = []
    orders = orderbook_df.to_dict(orient='records')

    if exchange == 'bybit':
        for order in orders:
            quotes.append({
                'price': order['price'],
                'quantity': order['quantity'],
                'order_amount': order['orderAmount'],
                'exchange': 'bybit',
            })
        return quotes
    elif exchange == 'gateio':
        for order in orders:
            quotes.append({
                'price': order['price'],
                'quantity': order['amount'],
                'order_amount': order['order_value'],
                'exchange': 'gateio',
            })
        return quotes
    else:
        raise ValueError('Exchange not supported.')


In [134]:
agg_bids = convert_orderbook_tick(bybit_bids, 'bybit') + convert_orderbook_tick(gateio_bids, 'gateio')
agg_asks = convert_orderbook_tick(bybit_asks, 'bybit') + convert_orderbook_tick(gateio_asks, 'gateio')

In [135]:
agg_bids_df = pd.DataFrame(agg_bids)
agg_asks_df = pd.DataFrame(agg_asks)

In [136]:
agg_bids_df.sort_values(by=['price'], ascending=False, inplace=True, ignore_index=True)
agg_asks_df.sort_values(by=['price'], ascending=True, inplace=True, ignore_index=True)

In [124]:
agg_bids_df

,price,quantity,order_amount,exchange
0,6.5,9.5,61.75,bybit
1,6.5,9.5,61.75,bybit
2,6.5,50,325,bybit
3,6.5,3,19.5,bybit
4,6.4,10,64,bybit
5,6.3,100,630,bybit
6,6.2,0.8,4.96,gateio
7,6.1,50,305,gateio
8,6.1,300,1830,bybit
9,6.1,100,610,gateio


In [125]:
agg_asks_df

,price,quantity,order_amount,exchange
0,6.99,8,55.92,gateio
1,6.99,6,41.94,gateio
2,6.99,5,34.95,gateio
3,6.99,4,27.96,gateio
4,6.99,3,20.97,gateio
5,6.99,2,13.98,gateio
6,6.99,1,6.99,gateio
7,7.01,4,28.04,gateio
8,7.03,2,14.06,gateio
9,7.36,1,7.36,gateio


In [144]:
pd.concat([
    agg_bids_df.rename(columns={c: f'{c}_bid' for c in agg_asks_df.columns}), 
    agg_asks_df.rename(columns={c: f'{c}_ask' for c in agg_asks_df.columns})], 
axis=1)

,price_bid,quantity_bid,order_amount_bid,exchange_bid,price_ask,quantity_ask,order_amount_ask,exchange_ask
0,6.5,9.5,61.75,bybit,6.99,8,55.92,gateio
1,6.5,50,325,bybit,6.99,6,41.94,gateio
2,6.5,3,19.5,bybit,6.99,5,34.95,gateio
3,6.5,9.5,61.75,bybit,6.99,4,27.96,gateio
4,6.4,10,64,bybit,6.99,3,20.97,gateio
5,6.3,100,630,bybit,6.99,2,13.98,gateio
6,6.2,0.8,4.96,gateio,6.99,1,6.99,gateio
7,6.1,50,305,gateio,7.01,4,28.04,gateio
8,6.1,300,1830,bybit,7.03,2,14.06,gateio
9,6.1,100,610,gateio,7.36,1,7.36,gateio


In [143]:
{f'{c}_bid' for c in agg_asks_df.columns}

{'exchange_bid', 'order_amount_bid', 'price_bid', 'quantity_bid'}